In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_csv('data/train.csv', index_col='ID')
test = pd.read_csv('data/test.csv', index_col='ID')

In [3]:
target = train.TARGET.values.copy()
train.drop('TARGET', axis=1, inplace=True)

In [4]:
train.loc[:, 'n0'] = (train == 0).sum(axis=1)
test.loc[:, 'n0'] = (test == 0).sum(axis=1)

In [5]:
dlist = list()
for c in train.columns:
    if train[c].unique().shape[0] == 1:
        print(c)
        dlist.append(c)
train.drop(dlist, axis=1, inplace=True)
test.drop(dlist, axis=1, inplace=True)

ind_var2_0
ind_var2
ind_var27_0
ind_var28_0
ind_var28
ind_var27
ind_var41
ind_var46_0
ind_var46
num_var27_0
num_var28_0
num_var28
num_var27
num_var41
num_var46_0
num_var46
saldo_var28
saldo_var27
saldo_var41
saldo_var46
imp_amort_var18_hace3
imp_amort_var34_hace3
imp_reemb_var13_hace3
imp_reemb_var33_hace3
imp_trasp_var17_out_hace3
imp_trasp_var33_out_hace3
num_var2_0_ult1
num_var2_ult1
num_reemb_var13_hace3
num_reemb_var33_hace3
num_trasp_var17_out_hace3
num_trasp_var33_out_hace3
saldo_var2_ult1
saldo_medio_var13_medio_hace3


In [6]:
for cone in train.columns:
    for ctwo in train.columns:
        if cone not in train.columns:
            continue
        if ctwo not in train.columns:
            continue
        if cone != ctwo and np.all(train[cone] == train[ctwo]):
            print(cone, ctwo)
            train.drop(ctwo, axis=1, inplace=True)
            test.drop(ctwo, axis=1, inplace=True)

ind_var6_0 ind_var29_0
ind_var6 ind_var29
ind_var13_medio_0 ind_var13_medio
ind_var18_0 ind_var18
ind_var26_0 ind_var26
ind_var25_0 ind_var25
ind_var32_0 ind_var32
ind_var34_0 ind_var34
ind_var37_0 ind_var37
ind_var40 ind_var39
num_var6_0 num_var29_0
num_var6 num_var29
num_var13_medio_0 num_var13_medio
num_var18_0 num_var18
num_var26_0 num_var26
num_var25_0 num_var25
num_var32_0 num_var32
num_var34_0 num_var34
num_var37_0 num_var37
num_var40 num_var39
saldo_var6 saldo_var29
saldo_var13_medio saldo_medio_var13_medio_ult1
delta_imp_reemb_var13_1y3 delta_num_reemb_var13_1y3
delta_imp_reemb_var17_1y3 delta_num_reemb_var17_1y3
delta_imp_reemb_var33_1y3 delta_num_reemb_var33_1y3
delta_imp_trasp_var17_in_1y3 delta_num_trasp_var17_in_1y3
delta_imp_trasp_var17_out_1y3 delta_num_trasp_var17_out_1y3
delta_imp_trasp_var33_in_1y3 delta_num_trasp_var33_in_1y3
delta_imp_trasp_var33_out_1y3 delta_num_trasp_var33_out_1y3


In [7]:
for c in train.columns:
    cmin = train[c].min()
    cmax = train[c].max()
    test.loc[test[c] < cmin, c] = cmin
    test.loc[test[c] > cmax, c] = cmax

In [8]:
import xgboost as xgb

In [9]:
from sklearn.cross_validation import StratifiedKFold

In [10]:
skf = StratifiedKFold(target, n_folds=8)

In [13]:
params = {'objective': 'binary:logistic', 
          'eval_metric': 'auc',
          'eta': 0.0202048,
          'max_depth': 5,
          'subsample': 0.6815,
          'colsample_bytree': 0.701,
          'silent': 1,
          'seed': 0
}

In [14]:
dtest = xgb.DMatrix(test)

In [15]:
preds_all = list()
kek = 1
for train_index, test_index in skf:
    print(kek)
    X_train, X_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    dtrain = xgb.DMatrix(X_train, y_train)
    dval = xgb.DMatrix(X_test, y_test)
    
    watchlist = ((dtrain, 'train'), (dval, 'val'))
    
    gbm = xgb.train(params, dtrain, num_boost_round=3000, evals=watchlist, maximize=True,
                    early_stopping_rounds=300)
    
    preds_x = gbm.predict(dtest, ntree_limit=gbm.best_ntree_limit)
    preds_all.append(preds_x)
    kek += 1

1
2
3
4
5
6
7
8


In [17]:
np.array(preds_all).mean(axis=0)

array([ 0.04595324,  0.05266068,  0.00267093, ...,  0.00418607,
        0.06014125,  0.00221829], dtype=float32)

In [18]:
sample = pd.read_csv('data/sample_submission.csv', index_col='ID')
sample.TARGET = np.array(preds_all).mean(axis=0)
sample.to_csv('submission/xgb_bag_cv.csv', index_label='ID')